In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd '../'

/Users/joe/Documents/boulot/git/homhots/HOTS


In [2]:
import numpy as np
import matplotlib.pyplot as plt

from HOTS.Event import Event, SimpleAlphabet
from HOTS.Layer import ClusteringLayer, FilterNHBD
from HOTS.ToolsMonitor import DisplayImage, DisplayHisto
from HOTS.Tools import SaveObject, LoadObject, GenerateHistogram
from HOTS.Classifier import Classifier
from HOTS.Network import Network
from HOTS.ToolsMonitor import DisplaySurface2D

In [3]:
tau = 0.02
R = 2
nb_cluster = [4, 8, 16]
ImageSize = (32, 32)

This notebook show classification on Simple Alphabet database

# Load Clustering, Training and Testing data

In [4]:
event_tr, event_te, event_cl, label_tr, label_te = SimpleAlphabet(
    NbTrainingData=36,
    NbTestingData=40,
    ClusteringData=[1, 32, 19, 22, 29],
    OutOnePolarity=False,
    verbose=0,
    Path="Data/alphabet_ExtractedStabilized.mat",
    LabelPath="Data/alphabet_label.pkl",
)

# Defining the Network 

In [5]:
InputFilter = FilterNHBD(threshold=15, neighbourhood=2)

L1 = ClusteringLayer(
    tau=tau,
    R=R,
    verbose=1,
    ThrFilter=0,
    LearningAlgo="lagorce",
    kernel="exponential",
    eta=None,
    eta_homeo=None,
    sigma=None,
)

L2 = ClusteringLayer(
    tau=10 * tau,
    R=2 * R,
    verbose=1,
    ThrFilter=0,
    LearningAlgo="lagorce",
    kernel="exponential",
    eta=None,
    eta_homeo=None,
    sigma=None,
)

L3 = ClusteringLayer(
    tau=10 * 10 * tau,
    R=2 * 2 * R,
    verbose=1,
    ThrFilter=0,
    LearningAlgo="lagorce",
    kernel="exponential",
    eta=None,
    eta_homeo=None,
    sigma=None,
)

Net = Network([InputFilter, L1, L2, L3])

# Training the Cluster on 5 images

In [6]:
ClusterLayer, event_output = Net.TrainCluster(
    event=event_cl, NbClusterList=nb_cluster, to_record=False, NbCycle=1
)

Generation of SpatioTemporal Surface in ------ 6.89 s
Clustering SpatioTemporal Surface in ------ 2.78 s
Generation of SpatioTemporal Surface in ------ 5.26 s
Clustering SpatioTemporal Surface in ------ 3.48 s
Generation of SpatioTemporal Surface in ------ 9.74 s
Clustering SpatioTemporal Surface in ------ 10.75 s


# Running the Network on training samples

In [7]:
event_o_tr = Net.RunNetwork(
    event_tr, NbClusterList=ClusterLayer, homrun=False, 
)

Generation of SpatioTemporal Surface in ------ 15.12 s
Generation of SpatioTemporal Surface in ------ 19.26 s
Generation of SpatioTemporal Surface in ------ 58.01 s


# Runing the Network on testing samples

In [8]:
event_o_te = Net.RunNetwork(event_te, NbClusterList=ClusterLayer, homrun=False)

Generation of SpatioTemporal Surface in ------ 18.76 s
Generation of SpatioTemporal Surface in ------ 22.54 s
Generation of SpatioTemporal Surface in ------ 46.86 s


# Classifying

In [9]:
Classif0 = Classifier(
    event_o_tr, event_o_te, TrainingLabel=label_tr, GroundTruth=label_te
)
prediction, accuracy, method = Classif0.HistogramDistance(to_print=True)

battacha:92.5% ### normalized:90.0% ### euclidian:95.0% ### 
